## Idea
- function turns `dataarray` with attached `hash` into new `dataarray` with attached `hash`
- `dataarray` can then be saved to `cache` directory

In [1]:
%autoreload
import numpy as np
import scipy.integrate as si
import xarray as xr

from xrcache import cached, keys, get_hash

In [2]:
ds = xr.Dataset({'bar': ('x', [1, 2, 3, 4]), 'x': list('abcd')})

hash_str = get_hash(ds.bar.data)

ds.attrs.update({keys.hash: hash_str})

ds

<xarray.Dataset>
Dimensions:  (x: 4)
Coordinates:
  * x        (x) <U1 'a' 'b' 'c' 'd'
Data variables:
    bar      (x) int64 1 2 3 4
Attributes:
    hash:     1074bd0a31dfaae87e1c96888a19f6589ac77cc2

In [3]:
@cached
def some_dataset_function(dataset: xr.Dataset) -> xr.Dataset:
    """add square of input"""
    ds = dataset.copy()
    result = dataset.bar ** 2
    new_ds = ds.update({"foo": result})

    return new_ds

some_dataset_function(ds)

<xarray.Dataset>
Dimensions:  (x: 4)
Coordinates:
  * x        (x) object 'a' 'b' 'c' 'd'
Data variables:
    bar      (x) int64 1 2 3 4
    foo      (x) int64 1 4 9 16
Attributes:
    hash:                e246cfc23dd64dc7d06f54de8dd37c787f7cae74
    hash_input:          1074bd0a31dfaae87e1c96888a19f6589ac77cc2
    function_signature:  {"__name__": "some_dataset_function", "dataset": nul...
    hash_function:       20138e90f9ca71952f7c136ebbcac0b0a495da2f

In [4]:
a = some_dataset_function(ds)
b = some_dataset_function(ds)

a.equals(b), a.attrs == b.attrs

(True, True)